# Import Libaries 

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv3D, MaxPool3D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import os
import datetime
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam
import cv2

In [5]:
import numpy as np
import SimpleITK as sitk
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [6]:
from glob import glob
from typing import  List

# Configure Model 

In [7]:
model = Sequential()
model.add(Conv3D(input_shape=(100,100,100,1),filters=20,kernel_size=3,padding="same", activation="relu"))
model.add(Conv3D(filters=64,kernel_size=3,padding="same", activation="relu"))
model.add(Dropout(0.4))
model.add(MaxPool3D(pool_size=2,strides=(2,2,2)))
model.add(Conv3D(filters=128, kernel_size=3, padding="same", activation="relu"))
model.add(Conv3D(filters=128, kernel_size=3, padding="same", activation="relu"))
model.add(Dropout(0.6))
model.add(MaxPool3D(pool_size=2,strides=(2,2,2)))
model.add(Conv3D(filters=256, kernel_size=3, padding="same", activation="relu"))
model.add(Conv3D(filters=256, kernel_size=3, padding="same", activation="relu"))
model.add(Conv3D(filters=256, kernel_size=3, padding="same", activation="relu"))
model.add(Conv3D(filters=256, kernel_size=3, padding="same", activation="relu"))
model.add(Dropout(0.8))
model.add(MaxPool3D(pool_size=2,strides=(2,2,2)))
model.add(Conv3D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Conv3D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Conv3D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Conv3D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Dropout(0.8))
model.add(MaxPool3D(pool_size=2,strides=(2,2,2)))
model.add(Conv3D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Conv3D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Conv3D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Conv3D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(Dropout(0.4))
model.add(MaxPool3D(pool_size=2,strides=(2,2,2)))

#Dense layer
model.add(Flatten())
#model.add(Dense(units=4096,activation="relu"))
#model.add(Dense(units=4096,activation="relu"))
#model.add(Dense(units=1000,activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 100, 100, 100, 20) 560       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 100, 100, 100, 64) 34624     
_________________________________________________________________
dropout (Dropout)            (None, 100, 100, 100, 64) 0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 50, 50, 50, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 50, 50, 128)   221312    
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 50, 50, 50, 128)   442496    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 50, 128)   0

In [9]:
model.load_weights('vgg19_3d_all_11012022_1210.h5') #Load Weights
print('Loading The Model')

Loading The Model


In [11]:
###---lAYER-Name--to-visualize--###
LAYER_NAME='conv3d_15'
# Create a graph that outputs target convolution and output
grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])
grad_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_input (InputLayer)    [(None, 100, 100, 100, 1) 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 100, 100, 100, 20) 560       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 100, 100, 100, 64) 34624     
_________________________________________________________________
dropout (Dropout)            (None, 100, 100, 100, 64) 0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 50, 50, 50, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 50, 50, 128)   221312    
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 50, 50, 50, 128)   442496

# Function to Resize the 3D volume 

In [12]:
def resize_image_with_crop_or_pad(image, img_size=(64, 64, 64), **kwargs):
    """Image resizing. Resizes image by cropping or padding dimension
     to fit specified size.
    Args:
        image (np.ndarray): image to be resized
        img_size (list or tuple): new image size
        kwargs (): additional arguments to be passed to np.pad
    Returns:
        np.ndarray: resized image
    """

    assert isinstance(image, (np.ndarray, np.generic))
    assert (image.ndim - 1 == len(img_size) or image.ndim == len(img_size)), \
        'Example size doesnt fit image size'

    # Get the image dimensionality
    rank = len(img_size)

    # Create placeholders for the new shape
    from_indices = [[0, image.shape[dim]] for dim in range(rank)]
    to_padding = [[0, 0] for dim in range(rank)]

    slicer = [slice(None)] * rank

    # For each dimensions find whether it is supposed to be cropped or padded
    for i in range(rank):
        if image.shape[i] < img_size[i]:
            to_padding[i][0] = (img_size[i] - image.shape[i]) // 2
            to_padding[i][1] = img_size[i] - image.shape[i] - to_padding[i][0]
        else:
            from_indices[i][0] = int(np.floor((image.shape[i] - img_size[i]) / 2.))
            from_indices[i][1] = from_indices[i][0] + img_size[i]

        # Create slicer object to crop or leave each dimension
        slicer[i] = slice(from_indices[i][0], from_indices[i][1])

    # Pad the cropped image to extend the missing dimension
    return np.pad(image[slicer], to_padding, **kwargs)

# Loading volume and compute the gradients 

In [13]:
def get_category(category, is_binary = True):
  result = np.zeros(2 if is_binary else 5)
  if is_binary:
    return 1 if category == 1 else 0
  elif category == 1:
    result[0] = 1
  else:
    result[category-1] = 1
  return result

In [23]:
import cv2
import SimpleITK as sitk
dimension = 100

def get_slices_3d_all(path, category):
    img = sitk.ReadImage(path, sitk.sitkFloat64)
    arr = sitk.GetArrayFromImage(img)
    
    slice = arr[30:130, : , :]
    
    arr = np.zeros([slice.shape[0], dimension,dimension])
    
    count = 0
    for i in range(arr.shape[0]):
        if arr[i:,:].max() != 0.0:
            print(count)
        arr[i, : , :] = cv2.resize(slice[i, : , :], (dimension, dimension), interpolation=cv2.INTER_CUBIC)
        arr[arr<0] = 0
        count += 1
    
    slices = np.array([arr])
    slices = slices.reshape(1,arr.shape[0], arr.shape[1], arr.shape[2], 1)
    slices_cat = np.array([get_category(category)])
    
    return slices, slices_cat

In [24]:
slices, slices_cat = get_slices_3d_all("./PREPROCESSED/BUY\\3062.nii.gz", 1)

In [25]:
slices.shape

(1, 100, 100, 100, 1)

In [41]:
###--Compute GRADIENT
from keras import backend as K
with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(slices)
    loss = predictions[:, 0]


# Extract filters and gradients
output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]

In [42]:
# Average gradients spatially
weights = tf.reduce_mean(grads, axis=(0, 1,2))
# Build a ponderated map of filters according to gradients importance
cam = np.zeros(output.shape[0:3], dtype=np.float32)

for index, w in enumerate(weights):
    cam += w * output[:, :, :, index]

In [43]:
img = sitk.ReadImage("./PREPROCESSED/BUY\\3062.nii.gz", sitk.sitkFloat64)
arr = sitk.GetArrayFromImage(img)


RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\1\sitk\Code\IO\src\sitkImageReaderBase.cxx:102:
sitk::ERROR: Unable to open "./PREPROCESSED/BUY\3062.nii.gz" for reading.

In [ ]:
arr.shape

In [ ]:
capi=resize(cam,arr.shape)
new_cams = np.empty(arr.shape)
for i in range(new_cams.shape[0]):
    cam_i = capi[i] - capi[i].mean()
    cam_i = (cam_i + 1e-10) / (np.linalg.norm(cam_i, 2) + 1e-10)
    #new_cams[i] = np.maximum(new_cams[i], 0)
    new_cams[i] = new_cams[i] / new_cams[i].max()

In [ ]:
new_cams[0]

In [ ]:
heatImg = sitk.GetImageFromArray(arr)
sitk.WriteImage(heatImg, "heatmap.nii.gz")

In [ ]:
from skimage.transform import resize
from matplotlib import pyplot as plt
capi=resize(cam,(80,100,100))
resized_img = slices
#print(capi.shape)
capi = np.maximum(capi,0)
heatmap = (capi - capi.min()) / (capi.max() - capi.min())
f, axarr = plt.subplots(2,3,figsize=(15,10));
f.suptitle('Grad-CAM')
slice_count= 55
slice_count2=30
    
axial_ct_img=np.squeeze(resized_img[0,slice_count, :,:,0])
axial_grad_cmap_img=np.squeeze(heatmap[slice_count,:, :])
print("axial ct", axial_ct_img.shape)
print("axial grad cam", axial_grad_cmap_img.shape)

img_plot = axarr[0,0].imshow(axial_ct_img, cmap='gray');
axarr[0,0].axis('off')
axarr[0,0].set_title('CT')
    
img_plot = axarr[0,1].imshow(axial_grad_cmap_img, cmap='jet');
axarr[0,1].axis('off')
axarr[0,1].set_title('Grad-CAM')
    

axial_overlay=cv2.addWeighted(axial_ct_img,0.5,axial_grad_cmap_img, 0.5, 0.0, dtype=cv2.CV_64F)
    
img_plot = axarr[0,2].imshow(axial_overlay,cmap='jet');
axarr[0,2].axis('off')
axarr[0,2].set_title('Overlay')

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(131)
plt.title('GradCAM')
plt.axis('off')
i = 60
plt.imshow(slices[0,i,:,:,0])
plt.imshow(heatmap[i,:,:], cmap='jet', alpha=0.5)

In [ ]:
copy = heatmap[heatmap == 0.0] = 1.0

In [ ]:
for i in range(heatmap.shape[0]):
    if heatmap[i,:,:].min() != 0.0:
        print(i, heatmap[i,:,:].min())

In [ ]:
heatmap.shape

In [ ]:
capi=resize(cam,(80,100,100))
resized_img = slices
#print(capi.shape)
#capi = np.maximum(capi,0)
heatmap = (capi - capi.min()) / (capi.max() - capi.min())

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
model = VGG16(include_top=True, weights='imagenet')

In [ ]:
model.summary()